# 

# Predictions

## Post-training analysis

In [7]:
import matplotlib as plt
import numpy as np
import pandas as pd
import seaborn as sns

filepath = f'../datasets/dataset_train.csv'
df = pd.read_csv(filepath)
df.drop(df.columns[2:6], inplace=True, axis = 1)
excluded_features = ["Arithmancy", "Defense Against the Dark Arts", "Care of Magical Creatures"]
df.drop(excluded_features, inplace=True, axis=1)
df.dropna(inplace=True)
target = 'Hogwarts House'
df_x_train = df[df.columns[2:]]
df_class = df[target]
df_x_train.head()

,Astronomy,Herbology,Divination,Muggle Studies,Ancient Runes,History of Magic,Transfiguration,Potions,Charms,Flying
0,-487.886086,5.727180,4.722,272.035831,532.484226,5.231058,1039.788281,3.790369,-232.79405,-26.89
1,-552.060507,-5.987446,-5.612,-487.340557,367.760303,4.107170,1058.944592,7.248742,-252.18425,-113.45
2,-366.076117,7.725017,6.140,664.893521,602.585284,3.555579,1088.088348,8.728531,-227.34265,30.42
3,697.742809,-6.497214,4.026,-537.001128,523.982133,-4.809637,920.391449,0.821911,-256.84675,200.64
4,436.775204,-7.820623,2.236,-444.262537,599.324514,-3.444377,937.434724,4.311066,-256.38730,157.98


In [9]:
from dslr.MultinomialTrainClass import LogRegTrain

training = LogRegTrain(df_x_train, df_class)
training.train(0.1, 100)

ModuleNotFoundError: No module named 'dslr'

In [ ]:
    print(self.x_train.shape, self.df_weights.shape)
    z = np.dot(self.x_train, self.df_weights)
    h = logreg.sigmoid(z)
    df_pred_proba = pd.DataFrame(h, columns=self.houses)
    df_pred_proba['Predicted outcome'] = df_pred_proba.idxmax(axis=1)
    df_pred_proba['Real outcome'] = self.df_class.tolist()
    accurate_pred = np.where(df_pred_proba['Predicted outcome']
                                == df_pred_proba['Real outcome'], 1, 0)
    df_pred_proba['Accurate pred.'] = accurate_pred